In [1]:
import cooltools
import cooler
from cooltools.eigdecomp import cis_eig
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cooltools'

In [ ]:
clr = cooler.Cooler('K562_hg19_chr3_50K.cool')
mat = clr.matrix(balance=True)[:]

The FPKM file contains gene expression levels
the H3K27me3 bigwig file contains ChIP-seq normalized signal
the GC file contains GC content data at 50Kb resolution.

In [ ]:
phasing = [] #phasing track is correcting for GC content

for line in open('hg19_GC_chr3_50K.txt'):
    line = line.rstrip()
    phasing.append(line)
    phasing_array = np.array(phasing)
    float_phasing_array = phasing_array.astype(float)
print(float_phasing_array)
print(type(float_phasing_array))

To capture the pattern of compartmentalization within-chromosomes, in cis, cooltools cis_eig first removes the dependence of contact frequency by distance, and then performs eigenedecompostion

Basically finding eigen vectors and values of this matrix

In [ ]:
cis_eigs = cooltools.eigdecomp.cis_eig(mat, n_eigs=1, phasing_track=float_phasing_array)
cis_eigs

#### Plot the first eigen vector, along with the interaction matrix such that you can see the relationship between eigen vector scores and compartment patterns in the Hi-C data (compartment scores over heatmap). You may need to play with the vmax argument in the matplotlib matshow function to get a good dynamic range of the data (this puts a cap on the maximum score displayed). I used ten times the median score of all of the nonzero interactions, but you can play around with this number if you like.

Hint: You can look at the examples in the documentation for ways to nicely combine the two plots.
Create a bed file from the first eigen vector with each ‘A’ and ‘B’ region as determined by the sign of the eigen vector score.

vmax = resolution; any score higher than x numbr will be cut off and predent it's lower than it actually is 

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
ax.matshow(mat, vmax=10) 
fig.show()


In [ ]:
plt.plot(np.arange(cis_eigs[1].shape[1]), cis_eigs[1][0,:])
plt.show()


In [ ]:
import matplotlib.gridspec as grd

plt.figure(figsize = (15, 15))

gs = grd.GridSpec(2, 2, height_ratios=[1,10], width_ratios=[6,1], wspace=1)
ax = plt.subplot(gs[2])
ax = ax.matshow(mat, aspect="auto", vmax=10)
ax2 = plt.subplot(gs[0])
ax2.plot(np.arange(cis_eigs[1].shape[1]), cis_eigs[1][0,:])
plt.axhline(y=0, color='r')
plt.show()

#### Create a bed file from the first eigen vector with each ‘A’ and ‘B’ region as determined by the sign of the eigen vector score.

In [ ]:
vector = cis_eigs[1][0] #just take vector, not value
vector.size #number of vals in array; each entry in eigenvector corresponds to a 50kb region in GC.txt file

In [ ]:
vector

In [ ]:
for i in range(30): #each value = 50kb of sequences
    print(vector[i])

In bedfile:
0       - 500,000 A(+)
500,000 - 550,000 B(-)

In [ ]:
positions = np.arange(0, 198050000, 50000) 
positions

In [ ]:
AB = []
filtered_pos = []
for i in range(len(vector)):
    if np.isnan(vector[i]):
        continue
    elif vector[i] > 0:
        AB.append("A")
        filtered_pos.append(positions[i])
    elif vector[i] < 0:
        AB.append("B")
        filtered_pos.append(positions[i])

#print(AB)
print(filtered_pos)
    

In [ ]:
sign_switch = []
for i in range(1, len(AB)):
    if AB[i] == AB[i-1]:
        continue
    else:
        sign_switch.append(i)
        
sign_switch
#len(sign_switch)
        

In [ ]:
start_list = [filtered_pos[0]]
for i in range(1, len(sign_switch)):
    start_list.append(filtered_pos[sign_switch[i-1]])
start_list
#len(start_list)

In [ ]:
print(start_list[0])
print(filtered_pos[1])

In [ ]:
AB

In [ ]:
f = open("bedfile.bed", "w")
for i in range(len(sign_switch)):
    f.write('\t'.join(["chr3", str(start_list[i]), str(filtered_pos[sign_switch[i]]), str(AB[sign_switch[i]-1])+'\n']))

f.close()
        


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
col_names= ["chr", "start", "end", "gene", "FPKM", "strand"]
df = pd.read_csv('K562_hg19_FPKM_chr3.bed', names=col_names, sep="\t", header=None)
df

In [ ]:
df["FPKM"] = np.log2(df["FPKM"]+1)
df

In [ ]:
hist = df.hist(column=["FPKM"], bins=50)
plt.axvline(x=1.6, ymax=1, color='r')

1) Adjust H3kme7 scores
2) Map H3 scores onto genes using bedtools map
3) Normalize gene length
4) Map onto compartments

$bedtools map -a K562_hg19_FPKM_chr3.bed -b K562_hg19_H3K27me3_chr3_norm.bg > me3_gene_map.bed

$awk 'BEGIN{OFS="\t"}{$7=$7 / ($3-$2); print $1,$2,$3,$4,$5,$6,$7}' me3_gene_map.bed > K562_hg19_FPKM_chr3_mapnorm.bed

$bedtools map -a K562_hg19_FPKM_chr3_mapnorm.bed -b bedfile.bed -o distinct -f 0.5 -c 4 > ABmapped.bed

-c default 5 parameters, switch to 4
-f Minimum overlap required as a fraction of A. Default is 1E-9 (i.e. 1bp).

FPKM = gene expression; over red line threshold

Need to subset out data for A-expressed (FPKM > 1.6), B-expressed, etc.)

In [ ]:
col_names2 = ["chr", "start", "end", "gene", "FPKM", "strand", "methylation", "compartment"]
ABmapped = pd.read_csv('ABmapped.bed', sep='\t', names = col_names2, header=None)
ABmapped

In [ ]:
A_expressed = ABmapped.loc[(ABmapped["FPKM"] > 1.6) & (ABmapped["compartment"] == "A")]
B_expressed = ABmapped.loc[(ABmapped["FPKM"] > 1.6) & (ABmapped["compartment"] == "B")]

In [ ]:
A_unexpressed = ABmapped.loc[(ABmapped["FPKM"] < 1.6) & (ABmapped["compartment"] == "A")]
B_unexpressed = ABmapped.loc[(ABmapped["FPKM"] < 1.6) & (ABmapped["compartment"] == "B")]

In [ ]:
print(A_expressed["methylation"])

In [ ]:
fig, ax = plt.subplots()

ax.violinplot([A_expressed["methylation"], B_expressed["methylation"], A_unexpressed["methylation"], B_unexpressed["methylation"]])

plt.xticks(ticks=[1, 2, 3, 4], labels = ["A expressed", "B expressed", "A unexpressed", "B unexpressed"])

ax.set_title('H3K27me3 levels by compartment and expression for chromosome 3')

plt.show()